# Code to Test the Critical Point Theory on Historical Data

This code was used to produce the results in sections 4.1 and 4.2.

1. Select a major earthquake and identify the time, epicenter location and magnitude

2. Select all earlier earthquakes in a circle of set radius around the epicenter

3. Exclude any earthquakes with a magnitude less than 2 points lower than the major earthquake magnitude

4. Calculate the cumulative Benioff strain for these earthquakes

5. Fit the data with a straight-line a power-law and a power-law with log-periodic oscillations

    (a) For a power-law fit the parameters m and B and use the values from the major earthquake for A (final cumulative Benioff strain) and t_{f} (time of the final earthquake)

    (b) The parameter m was restricted to be between 0 and 0.8 to ensure accelerating seismicity and the parameter B was restricted to be negative

    (c) The fitting was only carried out if there were at least 10 pre-cursor earthquakes

6. Calculate the RMSE for each fit and the ratio of a power-law to the straight-line RMSE values

7. Repeat for circles of increasing size (from 10 km up to 1300 km)

8. Find the radius with the smallest ratio i.e the one where a power-law fit is the best compared to the straight-line fit – this is the size of the critical region

In [ ]:
#Import required packages
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt, pi
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

import shapely.geometry
import pyproj
%matplotlib inline

## Prepare the Data

In [ ]:
#Read in the data - ANSS earthquake catalogue
df = pd.read_csv("Data/ANSS_catalogue.csv", encoding = 'latin')
print(df.shape)

#Convert the date to datetime and check the range
df.Date = pd.to_datetime(df.Date, format = '%d/%m/%Y')
print(df.Date.min())
print(df.Date.max())

#Add columns for the year, years since 1900 and days since 1900
df['year'] = df['Date'].dt.year
df['year_1900'] = df['year'] - 1900
df['day_1900'] = df['year_1900']*365 + df['Date'].dt.day

df.head(2)

In [ ]:
#Sort the dataset by magnitude
df.sort_values(['Magnitude'], ascending = False, inplace = True)

In [ ]:
'''
Remove aftershocks (rough method)
If more than one earthquake occurs on the same day
in the same approximate location keep only the largest earthquake
'''
df['lat_round'] = round(df.Latitude,0)
df['long_round'] = round(df.Longitude,0)
df['Date_round'] = df.year.map(str) + df['Date'].dt.week.map(str)
df.drop_duplicates(['Date_round','lat_round','long_round'],inplace = True)
df.drop(['lat_round','long_round', 'Date_round'], axis = 1, inplace = True)
print(df.shape)
df.head()

In [ ]:
#Calculate the energy release in Nm
df['Energy_log10'] = 4.8 + 1.5*df['Magnitude']
df['Energy'] = 10 ** df.Energy_log10
df.head(2)


In [ ]:
#Calculate the critical radius (from Jaume and Sykes 1999)
df['R_log10'] = -0.2 + 0.36*df['Magnitude']
df['R_km'] = 10 ** df.R_log10
df.head(2)

In [ ]:
#Drop some columns
df.drop(['Time','Depth','NbStations','Gap','Distance','RMS','Source','EventID'], axis = 1, inplace = True)

#Convert latlong from degrees to radians
df['lat_rad'] = (df.Latitude*pi)/180
df['lon_rad'] = (df.Longitude*pi)/180

In [ ]:
#Extract the key earthquakes from the supporting papers
key_earthquakes = df[~df.Paper.isna()]
key_earthquakes = key_earthquakes[~key_earthquakes.Name.isna()]
print(key_earthquakes.shape)
key_earthquakes

## Define Helper Functions

In [ ]:
"""
A function to calculate the great circle distance between two points 
on the earth (specified in radians)
This function was taken from the internet
"""
def haversine(target_lon, target_lat, all_lon, all_lat): 
    dlon = all_lon - target_lon 
    dlat = all_lat - target_lat 
    # haversine formula
    a = np.sin(dlat/2)**2 + np.cos(target_lat) * np.cos(all_lat) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6371 # Radius of earth in km. Use 3956 for miles
    return c * r

In [ ]:
"""
A Function to return all earthquakes within a radius of a target point
and with magnitude greater than M_min
"""
def get_points(target_point, all_points, radius = 100, M_min = 4):
        
    #Filter to only earthquakes over the set magnitude
    all_points = all_points[all_points.Magnitude >= M_min]   
    
    #Filter to only records before the date of the target point (and the target point itself)
    all_points = all_points[all_points.Date <= target_point.Date]
    #print("Number of earlier earthquakes: %d"  % all_points.shape[0])
    
    #Calculate the distance between the target point and all other points
    all_points['distance'] = haversine(target_point.lon_rad,target_point.lat_rad,
                                       all_points.lon_rad, all_points.lat_rad)
    
    #Filter to only points within the radius
    all_points = all_points[all_points['distance'] <= radius]
    #print("Number of earlier earthquakes within %d km: %d"  % (radius, all_points.shape[0]))
    
    #Calculate the time delta in days
    all_points['Time_Delta'] = (target_point.Date - all_points.Date).dt.days
    
    #Return the filtered dataset of earthquakes
    return(all_points)


In [ ]:
"""
Function to order a table of earthquakes by date 
and calculate their cummulative Benioff strain
"""
def benioff_calc(all_points):
    #Sort the earthquakes by date
    all_points.sort_values('Date', inplace = True)
    
    #Calculate the cummulative Benioff strain
    all_points['Energy_sqrt'] = np.sqrt(all_points.Energy)
    all_points['Cummulative_Benioff_Strain'] = all_points['Energy_sqrt'].cumsum()
    
    #Return the original dataset with a new column for the cummulative Benioff strain
    return(all_points)

In [ ]:
"""
Function to calculate the rmse for predictions
"""
def rmse_calc(pred, all_points):
    #Calculate rmse
    err = all_points.Cummulative_Benioff_Strain - pred
    err_squared = err**2
    mean_err = np.mean(err_squared)
    rmse = np.sqrt(mean_err)
    #Return the rmse
    return(rmse)

## Define Fitting Functions

In [ ]:
"""
Function to fit a straight-line to data
"""
def straight_line_fit(all_points, final_benioff):
    #Define the straight line function and parameters
    def straightLine(x, A, B):
        return A + B*x
    
    #Fit the data
    popt, pcov = curve_fit(straightLine, all_points.day_1900, all_points.Cummulative_Benioff_Strain)
    #Make predictions
    pred = popt[0] + popt[1]*all_points.day_1900
    #Calculate the rmse
    rmse = rmse_calc(pred, all_points) 
    return(popt, rmse)


In [ ]:
"""
Function to fit a power-law to data
"""
def power_law_fit(all_points, final_benioff, final_t):
    #Define the power-law function and parameters
    #Note that the parameters A and tf are fixed (taken from the major earthquake)
    def power_law(x, B, m):
        return B*x**m

    #Set a starting point for the parameter search
    starting_point = [-1000000, 0.3]
    #Fit the data
    popt, pcov = curve_fit(power_law,
                           final_t - all_points.day_1900, #x
                           all_points.Cummulative_Benioff_Strain - final_benioff,
                           p0 = starting_point,
                           maxfev = 10000,
                           bounds=([-np.inf,0], [np.inf,0.8]) #Set bounds on B and m
                          )
    #Make the predictions
    pred = final_benioff + popt[0]*(final_t - all_points.day_1900)**popt[1]
    #Calculate the rmse
    rmse = rmse_calc(pred, all_points)

    return(popt, rmse)



In [ ]:
"""
Function to fit a power-law with log-periodic corrections to data
"""
def lppl_fit(all_points, final_benioff, final_t):
    #Define the log-periodic power-law and parameters
    def lppl(x, omega, m, B, C, phi):
        return B*x**m + (B*C*x**m)*np.cos(omega * np.log(x) + phi)
    
    #Set a starting point for the parameters
    starting_point = [4.6,0.37, 8800000,-0.02, 4.08]
    #Fit the data
    popt, pcov = curve_fit(lppl, #function
                           final_t - all_points.day_1900, #x
                           final_benioff - all_points.Cummulative_Benioff_Strain, #A-e
                           maxfev=10000,
                           p0 = starting_point
                          )
    #Make predictions
    pred = final_benioff - popt[2]*(final_t - all_points.day_1900)**popt[1] + \
    -popt[2]*popt[3]*(final_t - all_points.day_1900)**popt[1] \
    *np.cos(popt[0] * np.log((final_t - all_points.day_1900)) + popt[4])

    #Calculate the rmse
    rmse = rmse_calc(pred, all_points)
    
    return(popt, rmse)


## Plotting Functions

In [ ]:
"""
Function to plot the points and fitted lines
"""
def plot_fit(sub_points, target_point, sl_opt, pl_opt, lppl_opt, lppl = False, method = 'Bowman'):
    
    #Set up an x-range
    x_range = range(sub_points.day_1900.min()-5,sub_points.day_1900.max()+5)
    x_range2 = [target_point.final_t - e for e in x_range]
    
    #Make the predictions
    sl_pred = sl_opt[0] + sl_opt[1]*x_range
    pl_pred = target_point.final_benioff + pl_opt[0]*x_range2**pl_opt[1]
    if(lppl):
        lppl_pred = target_point.final_benioff - lppl_opt[2]*x_range2**lppl_opt[1] + \
        -lppl_opt[2]*lppl_opt[3]*x_range2**lppl_opt[1]*np.cos(lppl_opt[0] * np.log(x_range2) + lppl_opt[4])
    
    #Make the plot
    fig, ax = plt.subplots()
    
    #Plot the cummulative Benioff strain
    ax.plot(sub_points.day_1900/365, sub_points.Cummulative_Benioff_Strain, marker='o', ls = 'None')
    #ax.set_xlim(sub_points.Time_Delta.max()/365 + 5, 0)  # decreasing time
    ax.set_xlabel('Years since 1900')
    ax.set_ylabel('Cumulative Benioff Strain (J^1/2)')
    ax.set_title('%s Latlong (%.2f, %.2f), r = %dkm' % (target_point.Name,
                                                        target_point.Latitude,
                                                        target_point.Longitude,
                                                        target_point.r))
    ax.grid(True)
    
    #Plot the fitted straight line and power law
    x_range_plot = [e/365 for e in x_range]
    ax.plot(x_range_plot, sl_pred, ls = '--', label='Straight-line fit')
    ax.plot(x_range_plot, pl_pred, ls = '--', label='Power-law fit')
    
    ax.legend()
    
    #If required, plot the LPPL fit
    if(lppl):
        ax.plot(x_range_plot, lppl_pred, ls = '--', label='LPPL fit')
        ax.legend()
        plt.savefig('Fitted_Data/' + method + '/' + target_point.Name + '_' + str(round(target_point.r,0)) + '_lppl_fits.png')
    else:
        plt.savefig('Fitted_Data/' + method + '/' + target_point.Name + '_' + str(round(target_point.r,0)) + '_fits.png')
    plt.show()

In [ ]:
"""
Function to plot the c-values (curvature parameter) from the Bowman method
"""
def c_plot(c_values, lat, lon, min_r, min_c, name, lppl = False, method = 'Bowman'):
    #Plot the values
    fig, ax = plt.subplots()
    ax.plot(*zip(*c_values), marker = 'o')
    ax.set_xlabel('Region radius (km)')
    
    #Set the title depending on whether C or C2 has been plotted
    if(lppl):
        ax.set_ylabel('C2 (lppl rmse / power-law rmse)')
        ax.set_title('%s Latlong (%.2f, %.2f), C2 values' % (name, lat, lon))
    else:
        ax.set_ylabel('C (power-law rmse / straight-line rmse)')
        ax.set_title('%s Latlong (%.2f, %.2f), C values' % (name, lat, lon))
    ax.grid(True)
    
    #Save out the plot
    file_name = 'Fitted_Data/' + method + '/' + name
    if(lppl):
        plt.savefig(file_name +  '_lppl_c.png')
    else:
        plt.savefig(file_name +  '_c.png')

In [ ]:
"""
Function to plot the c-values (curvature parameter) from the Robinson method
In this case we hold two values from the radius, lat delta and long delta steady
and vary the third for the plot
"""
def c_plot_robinson(c_values, lat, lon, fixed_ref, name, lppl = False, method = 'Robinson'):
    all_refs = ['Radius (km)', 'Lat (delta)', 'Long (delta)']
    
    #Find the two varying values
    varying_refs = [e for e in all_refs if e != fixed_ref]
    
    #Plot the curvature parameter for the varying value
    fig, ax = plt.subplots()
    ax.plot(*zip(*c_values), marker = 'o')
    ax.set_xlabel(fixed_ref)
    
    #Set the y-axis title and graph titile
    if(lppl):
        ax.set_ylabel('C2 (lppl rmse / power-law rmse)')
        ax.set_title('%s, fixed %s and %s, varying %s, C2 values' % (name,
                                                                     varying_refs[0],
                                                                     varying_refs[1], fixed_ref))
    else:
        ax.set_ylabel('C (power-law rmse / straight-line rmse)')
        ax.set_title('%s, varying %s, C values' % (name, fixed_ref))
    ax.grid(True)
    
    #Save out the plot
    file_name = 'Fitted_Data/' + method + '/' + name
    if(lppl):
        plt.savefig(file_name + '_' + fixed_ref +  '_lppl_c.png')
    else:
        plt.savefig(file_name + '_' + fixed_ref +  '_c.png')
    

## Fitting Master Function

In [ ]:
"""
Function to fit the data using the different methods (straight-line and power-law)
Returns the dataset used for the fit, the curvature parameter 
and the power law fitting parameters
"""
def fitting_function(target_point, df, r, lat_delta = 0, long_delta = 0,
                     make_plot = False, return_data = False,
                     method = 'Bowman', master_lppl = True):
    
    target_point['r'] = r
    #Get the relevant earthquakes within the radius
    sub_points = get_points(target_point, df, radius = r,
                                        M_min = target_point.Magnitude - 2,
                                        lat_delta = lat_delta, long_delta = long_delta)
    
    #Check if there are enough points to continue with the fit
    if(sub_points.shape[0] < 10):
        return

    #Calculate the cummulative benioff strain
    sub_points = benioff_calc(sub_points)
    target_point['final_benioff'] = sub_points.iloc[sub_points.shape[0]-1].Cummulative_Benioff_Strain
    target_point['final_t'] = sub_points.iloc[sub_points.shape[0]-1].day_1900

    #Remove earthquakes on the final day from the table
    sub_points = sub_points[sub_points.day_1900 < target_point['final_t']]

    #sub_points.drop(sub_points.tail(1).index,inplace=True)
    sub_points.index = sub_points.day_1900

    #Check if there are enough points to continue with the fit
    if(sub_points.shape[0] < 6):
        return

    #Fit to a straight line
    sl_opt, sl_rmse = straight_line_fit(sub_points,
                                        target_point['final_benioff'])

    #Fit to a power law straight line
    pl_opt, pl_rmse = power_law_fit(sub_points,
                                    target_point['final_benioff'],
                                    target_point['final_t'])
    #Calculate C
    c = pl_rmse/sl_rmse
    
    #If required, try fitting to the LPPL
    if(master_lppl):
        try:
            #Fit to the lppl
            lppl_opt, lppl_rmse = lppl_fit(sub_points,
                                           target_point['final_benioff'],
                                           target_point['final_t'])
            c2 = lppl_rmse/pl_rmse
        except:
            lppl_opt = None
            c2 = None
            master_lppl = False
    else:
        lppl_opt = None
        c2 = None
        
    #Plot the cummulative Benioff strain and the fitted lines if required
    if(make_plot):
        #Set the latitude and longitude used
        target_point.Latitude = round(target_point.Latitude + lat_delta*180/pi,2)
        target_point.Longitude = round(target_point.Longitude + long_delta*180/pi,2)
        plot_fit(sub_points, target_point, sl_opt, pl_opt, lppl_opt, master_lppl, method)
        
    #If required, return the data used for the fit and the fitted parameters
    if(return_data):
        return(sub_points, sl_opt, pl_opt, lppl_opt)    
    
    return(r, c, c2)

## Apply the Bowman Method

In [ ]:
"""
Loop through all key earthquakes and apply the Bowman method
"""
#Set up the dataframe to store results
bowman_results = pd.DataFrame(columns=['id', 'bowman_r', 'bowman_c', 'bowman_r_lppl', 'bowman_c_lppl'])

for i in range(0,key_earthquakes.shape[0]):
    #Set the target point
    target_point = key_earthquakes.iloc[i].copy()
    name = target_point.Name
    print(name)
    print("Target Point lat, long, magnitude: (%f, %f, %f)" % \
          (target_point.Latitude, target_point.Longitude, target_point.Magnitude))

    c_values = []
    c2_values = []
    #Loop through the different radii
    for r in np.arange(10, 1300, 10).tolist():
        #Apply the master fitting function
        result = fitting_function(target_point, df, r, make_plot = True)
        if(result == None):
            continue
        r, c, c2 = result
        #Store the resulting curvature parameters
        c_values.append((r, c))
        if(c2 != None):
            c2_values.append((r, c2))
     
    if(len(c_values) == 0):
        continue
    #Find the radius at the optimum curvature parameter (C)
    min_r, min_c = min(c_values, key = lambda t: t[1])
    print("Minimum c value %.2f, occurs at a radius of %dkm" % (min_c, min_r))
    #Plot the variation in the curvature parameter with radius
    c_plot(c_values, target_point.Latitude, target_point.Longitude, min_r,min_c, name)
    
    #Find the radius at the optimum curvature parameter (C2)
    min_r_lppl, min_c_lppl = min(c2_values, key = lambda t: t[1])
    print("Minimum c value lppl %.2f, occurs at a radius of %dkm" % (min_c_lppl, min_r_lppl))
    #Plot the variation in C2 with radius
    c_plot(c2_values, target_point.Latitude, target_point.Longitude,
       min_r_lppl,min_c_lppl, name, lppl = True)
    
    #Add to the results table
    bowman_results = bowman_results.append({'id': target_point.event_id,
                           'bowman_r': min_r,
                           'bowman_c': min_c,
                           'bowman_r_lppl': min_r_lppl,
                           'bowman_c_lppl': min_c_lppl}, 
                          ignore_index=True)
bowman_results

In [ ]:
#Save out the results
bowman_results.to_csv('Fitted_Data/fitting_results_bowman_3.csv', index = False)


In [ ]:

"""
Loop through all sample earthquakes using the Robinson method
Use the Jaume and Sykes method to calculate R (critical region radius)
Vary this by +/-25% in steps of 5%
Vary the latlong by +/-0.5 degrees in steps of 0.1 degree 
"""
#Set up a dataframe to store the results
robinson_results = pd.DataFrame(columns=['id', 'robinson_r','robinson_lat_delta','robinson_long_delta',
                                         'robinson_c', 'robinson_r_lppl',
                                         'robinson_lat_delta_lppl','robinson_long_delta_lppl',
                                         'robinson_c_lppl'])

for i in range(0,key_earthquakes.shape[0]):
    #Define the target point
    target_point = key_earthquakes.iloc[i].copy()
    name = target_point.Name
    print(name)
    print("Target Point lat, long, magnitude, radius: (%f, %f, %f, %f)" % \
          (target_point.Latitude, target_point.Longitude, target_point.Magnitude, target_point.R_km))

    c_values = []
    c2_values = []

    #Loop through all epicentres and radii
    for r_delta in np.arange(-0.25, 0.25, 0.05):
        r = (1 + r_delta)*target_point.R_km
        for lat_delta in np.arange(-0.5, 0.6, 0.1).tolist():
            lat_delta = np.round(lat_delta, 2)*pi/180
            for long_delta in np.arange(-0.5, 0.6, 0.4).tolist():
                long_delta = np.round(long_delta, 2)*pi/180

                #Apply the master fitting function
                result = fitting_function(target_point, df, r, lat_delta, long_delta, make_plot = True)
                if(result == None):
                    continue
                r, c, c2 = result
                #Store the resulting curvature parameters
                c_values.append((r, lat_delta, long_delta, c))
                if(c2 != None):
                    c2_values.append((r, lat_delta, long_delta, c2))
                
    if(len(c_values) == 0):
        continue

    #Find r, lat delta and long delta values at the optimum curvature parameters
    min_r, min_lat_delta, min_long_delta, min_c = min(c_values, key = lambda t: t[3])
    min_r_lppl, min_lat_delta_lppl, min_long_delta_lppl, min_c_lppl = min(c2_values, key = lambda t: t[3])
    print("Minimum c value %.2f, occurs at a radius of %dkm" % (min_c, min_r))
    print("Minimum c lppl value %.2f, occurs at a radius of %dkm" % (min_c_lppl, min_r_lppl))
    
    #Plot the variation in C with r, lat and long - vary one at a time
    fix_loc_c = [(e[0],e[3]) for e in c_values if (e[1] == min_lat_delta) & (e[2] == min_long_delta)]
    fix_rlat_c = [(e[2],e[3]) for e in c_values if (e[1] == min_lat_delta) & (e[0] == min_r)]
    fix_rlong_c = [(e[1],e[3]) for e in c_values if (e[0] == min_r) & (e[2] == min_long_delta)]
    fix_loc_c_lppl = [(e[0],e[3]) for e in c2_values if (e[1] == min_lat_delta_lppl) & (e[2] == min_long_delta_lppl)]
    fix_rlat_c_lppl = [(e[2],e[3]) for e in c2_values if (e[1] == min_lat_delta_lppl) & (e[0] == min_r_lppl)]
    fix_rlong_c_lppl = [(e[1],e[3]) for e in c2_values if (e[0] == min_r_lppl) & (e[2] == min_long_delta_lppl)]
    all_sub = [fix_loc_c, fix_rlat_c, fix_rlong_c, fix_loc_c_lppl, fix_rlat_c_lppl, fix_rlong_c_lppl]
    all_ref = ['Radius (km)', 'Lat (delta)', 'Long (delta)',
               'Radius (km)', 'Lat (delta)', 'Long (delta)']
    
    for i in [0,1,2]:
        c_plot_robinson(all_sub[i], target_point.Latitude, target_point.Longitude, all_ref[i],
                        name, lppl = False, method = 'Robinson')
    for i in [3,4,5]:
        c_plot_robinson(all_sub[i], target_point.Latitude, target_point.Longitude,all_ref[i],
                        name, lppl = True, method = 'Robinson')
    
    #Add the results to the dataframe
    robinson_results = robinson_results.append({'id': target_point.event_id,
                                                'robinson_r': min_r,
                                                'robinson_lat_delta': min_lat_delta,
                                                'robinson_long_delta': min_long_delta,
                                                'robinson_c': min_c,
                                                'robinson_r_lppl': min_r_lppl,
                                                'robinson_lat_delta_lppl': min_lat_delta_lppl,
                                                'robinson_long_delta_lppl': min_long_delta_lppl,
                                                'robinson_c_lppl': min_c_lppl}, 
                                               ignore_index=True)

robinson_results

In [ ]:
#Save out the results
robinson_results.to_csv('Fitted_Data/fitting_results_robinson_3.csv', index = False)


# Combine the Results from both Methods


In [ ]:
#Combine the tables and save out
results = bowman_results.merge(robinson_results, how = 'left', on = 'id')
results.to_csv('Fitted_Data/fitting_results_comparison_4.csv', index = False)
results.head()

In [ ]:
"""
Loop through all earthquakes in the results table
Rerun the fit for the optimum radius, lat and long
Save out the fit graphs and the parameters
"""
#Set up a dataframe to store the results
results2 = pd.DataFrame(columns = list(results.columns) + ['bowman_pl_m', 'bowman_pl_B',
                                                           'bowman_lppl_m','bowman_lppl_B',
                                                           'bowman_lppl_omega', 'bowman_lppl_C',
                                                           'bowman_lppl_phi',
                                                          'robinson_pl_m', 'robinson_pl_B',
                                                           'robinson_lppl_m','robinson_lppl_B',
                                                           'robinson_lppl_omega', 'robinson_lppl_C',
                                                          'robinson_lppl_phi'])

#Loop through all earthquakes
for i in range(0,results.shape[0]):
    #Define the target point
    target_point = key_earthquakes[key_earthquakes.event_id == results.id.iloc[i]].iloc[0]
    name = target_point.Name
    print(name)
    print("Target Point lat, long, magnitude, radius: (%f, %f, %f, %f)" % \
          (target_point.Latitude, target_point.Longitude, target_point.Magnitude, target_point.R_km))
    
    #Run the fitting for Bowman
    bowman_fit = fitting_function(target_point, df, results.bowman_r.iloc[i],
                                  make_plot = True, return_data = True,
                                  method = "Bowman", master_lppl = False)
    
    #Run the fitting for Robinson
    robinson_fit = fitting_function(target_point, df, results.robinson_r.iloc[i],
                                    results.robinson_lat_delta.iloc[i],
                                    results.robinson_long_delta.iloc[i],
                                   make_plot = True, return_data = True,
                                    method = "Robinson", master_lppl = False)
    
    #Run the fitting for Bowman - LPPL
    bowman_fit_lppl = fitting_function(target_point, df, results.bowman_r_lppl.iloc[i],
                                  make_plot = True, return_data = True,
                                  method = "Bowman", master_lppl = True)
    
    #Run the fitting for Robinson - LPPL
    robinson_fit_lppl = fitting_function(target_point, df, results.robinson_r_lppl.iloc[i],
                                    results.robinson_lat_delta_lppl.iloc[i],
                                    results.robinson_long_delta_lppl.iloc[i],
                                   make_plot = True, return_data = True,
                                    method = "Robinson", master_lppl = True)
    
    #Add the fitted parameters to the table    
    results_dict = dict(results.iloc[i])
    results_dict['bowman_pl_m'] = bowman_fit[2][1]
    results_dict['bowman_pl_B'] = bowman_fit[2][0]
    results_dict['bowman_lppl_m'] = bowman_fit_lppl[3][1]
    results_dict['bowman_lppl_B'] = -bowman_fit_lppl[3][2]
    results_dict['bowman_lppl_omega'] = bowman_fit_lppl[3][0]
    results_dict['bowman_lppl_C'] = bowman_fit_lppl[3][3]
    results_dict['bowman_lppl_phi'] = bowman_fit_lppl[3][4]
    
    if(robinson_fit != None):
        results_dict['robinson_pl_m'] = robinson_fit[2][1]
        results_dict['robinson_pl_B'] = robinson_fit[2][0]
        results_dict['robinson_lppl_m'] = robinson_fit_lppl[3][1]
        results_dict['robinson_lppl_B'] = -robinson_fit_lppl[3][2]
        results_dict['robinson_lppl_omega'] = robinson_fit_lppl[3][0]
        results_dict['robinson_lppl_C'] = robinson_fit_lppl[3][3]
        results_dict['robinson_lppl_phi'] = robinson_fit_lppl[3][4]
    
    results2 = results2.append(results_dict, ignore_index=True)
results2 

In [ ]:
#Combine the optimum values/parameters table with the original dataset
final_results = df.merge(results2, how = 'inner', left_on = 'event_id', right_on = 'id')
final_results.head(2)

In [ ]:
#Format the dataframe 
final_results_formatted = final_results.copy()
final_results_formatted.drop(['MagType','event_id', 'year', 'year_1900',
                              'day_1900', 'Energy_log10','Energy', 'R_log10',
                             'lat_rad', 'lon_rad', 'id','robinson_r_lppl',
                             'robinson_lppl_B','robinson_lat_delta_lppl',
                             'robinson_long_delta_lppl', 'robinson_c_lppl',
                             'robinson_lppl_m', 'robinson_lppl_B', 'robinson_lppl_omega',
                              'robinson_lppl_C', 'robinson_lppl_phi'],
                             axis = 1, inplace = True)
final_results_formatted = final_results_formatted.round(2)

final_results_formatted[['R_km', 'bowman_r', 'bowman_r_lppl',
                         'robinson_r','bowman_pl_B','bowman_lppl_B',
                         'robinson_pl_B']] = \
                final_results_formatted[['R_km', 'bowman_r', 'bowman_r_lppl',
                                         'robinson_r','bowman_pl_B','bowman_lppl_B',
                                        'robinson_pl_B']].fillna(0.0).astype(int)
    
final_results_formatted.columns = ['Date', 'Lat', 'Long', 'Mag', 'Name', 'Paper',
                                   'Calculated R (km)','bowman_r', 'bowman_c',
                                   'bowman_r_lppl', 'bowman_c_lppl','robinson_r',
                                   'robinson_lat_delta', 'robinson_long_delta', 'robinson_c',
                                   'bowman_pl_m',
                                   'bowman_pl_B', 'bowman_lppl_m', 'bowman_lppl_B',
                                   'bowman_lppl_omega','bowman_lppl_C', 'bowman_lppl_phi',
                                   'robinson_pl_m', 'robinson_pl_B']

#Rearrange the columns
final_results_formatted = final_results_formatted[['Date', 'Lat', 'Long', 'Mag', 'Name', 'Paper',
                            'Calculated R (km)','bowman_r','robinson_r', 'bowman_c',
                            'robinson_c','bowman_pl_m','robinson_pl_m','bowman_pl_B',
                            'robinson_pl_B','robinson_lat_delta', 'robinson_long_delta',
                            'bowman_r_lppl', 'bowman_c_lppl','bowman_lppl_m', 'bowman_lppl_B',
                            'bowman_lppl_omega','bowman_lppl_C', 'bowman_lppl_phi']]
final_results_formatted.head(2)

In [ ]:
#Save out the final results table
final_results_formatted.to_csv('Fitted_Data/formatted_results_2.csv', index = False)
